In [30]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [50]:
import itertools
import random

#### Generators and `throw()`

In [47]:
class Error(Exception):
    pass


def gen():
    
    yield 1
    try:
        yield 2
    except Error as err:
        print(repr(err))
    else:
        yield 3
    yield 4
    
it = gen()
next(it)
next(it)
it.throw(Error("test error!"))

1

2

Error('test error!')


4

In [272]:
class Reset(Exception):
    pass


def timer(period):
    try:
        current = period
        while current:
            current -= 1
            try:
                yield current
            except Reset:
                current = period
    finally:
        pass

def poll_for_reset(b):
    rnd = random.randrange(b)
    if rnd == b-1:
        return True
    return False


def run():
    it = timer(4)
    is_started = False
    while True:
        try:
            if is_started and poll_for_reset(5):
                current = it.throw(Reset())
            else:
                is_started = True
                current = next(it)
        except StopIteration:
            break
        else:
            print(f"remaining: {current}")
            
run()

remaining: 3
remaining: 2
remaining: 1
remaining: 0


#### Itertools

In [276]:
it = itertools.chain([1, 2, 3], [4, 5])
list(it)

[1, 2, 3, 4, 5]

In [277]:
it = itertools.repeat("hi", 5)
list(it)

['hi', 'hi', 'hi', 'hi', 'hi']

In [278]:
it = itertools.cycle([4, "a"])
[next(it) for _ in range(10)]

[4, 'a', 4, 'a', 4, 'a', 4, 'a', 4, 'a']

In [282]:
it1, it2 = itertools.tee([1, 2, 3], 2)
next(it1)
next(it2)
next(it1)
next(it2)

1

1

2

2

In [285]:
it = itertools.zip_longest([1, 2], "a", fillvalue="foo")
list(it)

[(1, 'a'), (2, 'foo')]

In [295]:
odds = itertools.islice(range(10), 1, 10, 2)
list(odds)

[1, 3, 5, 7, 9]

In [324]:
it = itertools.product([1, 2], repeat=2)
list(it)
it = itertools.product([1, 2], "ab")
list(it)
it = itertools.product([1, 2], "ab", repeat=2)
list(it)

[(1, 1), (1, 2), (2, 1), (2, 2)]

[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b')]

[(1, 'a', 1, 'a'),
 (1, 'a', 1, 'b'),
 (1, 'a', 2, 'a'),
 (1, 'a', 2, 'b'),
 (1, 'b', 1, 'a'),
 (1, 'b', 1, 'b'),
 (1, 'b', 2, 'a'),
 (1, 'b', 2, 'b'),
 (2, 'a', 1, 'a'),
 (2, 'a', 1, 'b'),
 (2, 'a', 2, 'a'),
 (2, 'a', 2, 'b'),
 (2, 'b', 1, 'a'),
 (2, 'b', 1, 'b'),
 (2, 'b', 2, 'a'),
 (2, 'b', 2, 'b')]

In [331]:
cashflows = [1000, -90, -90, -90, -90]
list(itertools.accumulate(cashflows, lambda bal, pmt: bal*1.05 + pmt))

[1000, 960.0, 918.0, 873.9000000000001, 827.5950000000001]

In [336]:
it = itertools.dropwhile(lambda x: x ** 2 < 10, range(10))
list(it)

[4, 5, 6, 7, 8, 9]

In [348]:
it = itertools.permutations("abc", 2)
list(it)

[('a', 'b'), ('a', 'c'), ('b', 'a'), ('b', 'c'), ('c', 'a'), ('c', 'b')]

In [351]:
it = itertools.combinations_with_replacement([1, 2, 3], 2)
for el in it:
    print(el)

(1, 1)
(1, 2)
(1, 3)
(2, 2)
(2, 3)
(3, 3)


#### Properties

In [24]:
from datetime import timedelta, datetime


class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0

    def __repr__(self):
        return (f'NewBucket(max_quota={self.max_quota}, '
                f'quota_consumed={self.quota_consumed})')
    
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # Quota being reset for a new period
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # Quota being filled for the new period
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
           # Quota being consumed during the period
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed = delta

def fill(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount


def deduct(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        return False  # Bucket hasn't been filled this period
    if bucket.quota - amount < 0:
        return False  # Bucket was filled, but not enough

    bucket.quota -= amount
    return True       # Bucket had enough, quota consumed
        
bucket = NewBucket(60)
print('Initial', bucket)
fill(bucket, 100)
print('Filled', bucket)

if deduct(bucket, 50):
    print('Had 50 quota')
else:
    print('Not enough for 50 quota')
print('Now', bucket)

if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')
print('Now', bucket)
if deduct(bucket, 1):
    print('Had 1 quota')
else:
    print('Not enough for 1 quota')
print('Now', bucket)
print('Still', bucket)

Initial NewBucket(max_quota=0, quota_consumed=0)
Filled NewBucket(max_quota=100, quota_consumed=0)
Had 50 quota
Now NewBucket(max_quota=100, quota_consumed=50)
Had 3 quota
Now NewBucket(max_quota=100, quota_consumed=53)
Had 1 quota
Now NewBucket(max_quota=100, quota_consumed=54)
Still NewBucket(max_quota=100, quota_consumed=54)


#### Descriptors

In [61]:
from weakref import WeakKeyDictionary

In [62]:
class Grade(object):
    
    def __init__(self):
        self._values = WeakKeyDictionary()
        
    def __get__(self, instance, owner):
        return self._values.get(instance, None)
    
    def __set__(self, instance, value):
        if not 0 <= value <= 100:
            raise ValueError(f"Grade should be between 0 and 100, got {repr(value)} instead")
        self._values[instance] = value
    

class Exam(object):
    """Hold grades for all exams."""
    math = Grade()

    
exam = Exam()
exam.math = 75
print(exam.math)

exam1 = Exam()
exam1.math = 99
print(exam.math, exam1.math)

# Grade() constructed once for a class dictionary

75
75 99


#### Use \_\_getattr\_\_, \_\_getattribute\_\_, and \_\_setattr\_\_ for Lazy Attributes

In [83]:
class SavingRecord:
    
    
    def __getattribute__(self, name):
        print(f'* Called __getattribute__({name!r})')
        return super().__getattribute__(name)
    
    def __setattr__(self, name, value):
        # Save some data for the record
        object.__setattr__(self, name, value)
        
record = SavingRecord()
record.a = 4
record.a

* Called __getattribute__('a')


4

In [91]:
class DictionaryRecord:
    def __init__(self, data):
        self._data = data

    def __getattribute__(self, name):
        print(f'* Called __getattribute__({name!r})')
        data_dict = super().__getattribute__('_data')
        return data_dict[name]
    
data = DictionaryRecord({"a": 3})
data.a

* Called __getattribute__('a')


3

#### \_\_init\_subclass\_\_

In [97]:
class Philosopher:
    def __init_subclass__(cls, /, default_name, **kwargs):
        print(kwargs)
        super().__init_subclass__(**kwargs)
        cls.default_name = default_name

class AustralianPhilosopher(Philosopher, default_name="Bruce"):
    pass

phil = AustralianPhilosopher()
phil.default_name

{}


'Bruce'

In [99]:
class Top:
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Top for {cls}')

class Left(Top):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Left for {cls}')

class Right(Top):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Right for {cls}')

class Bottom(Left, Right):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Bottom for {cls}')
        
class B(Bottom):
    pass

Top for <class '__main__.Left'>
Top for <class '__main__.Right'>
Top for <class '__main__.Bottom'>
Right for <class '__main__.Bottom'>
Left for <class '__main__.Bottom'>
Top for <class '__main__.B'>
Right for <class '__main__.B'>
Left for <class '__main__.B'>
Bottom for <class '__main__.B'>
